In [1]:
import tensorflow as tf

import pkg_resources as pkg_rs
tensorflow_version = pkg_rs.get_distribution("tensorflow").version
print tensorflow_version

0.9.0rc0


### installation directory

/Users/peterhirt/anaconda/envs/tf/lib/python2.7/site-packages/tensorflow

In [2]:
import cPickle as pickle
import numpy as np

In [3]:
flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string('summaries_dir', '../tensorboard1', 'Summaries directory')

# launch tensorboard with
# tensorboard --logdir='/Users/peterhirt/datascience/ild-cnn/tensorboard1'

if the tensorboard data directory exists, delete the content and create an empty directory

In [4]:
if tf.gfile.Exists(FLAGS.summaries_dir):
    tf.gfile.DeleteRecursively(FLAGS.summaries_dir)
tf.gfile.MakeDirs(FLAGS.summaries_dir)

# load ILD_CNN dataset

both training set and validation set is loaded, normalised and the labels hot-encoded.

In [5]:
def load_data():

    # load the dataset as X_train and as a copy the X_val
    # the labels are loaded as one-hot encoded data
    
    X_train = pickle.load( open( "../pickle/X_train.pkl", "rb" ) )
    y_train = pickle.load( open( "../pickle/y_train_hot.pkl", "rb" ) )
    X_val = pickle.load( open( "../pickle/X_val.pkl", "rb" ) )
    y_val = pickle.load( open( "../pickle/y_val_hot.pkl", "rb" ) )
    
    X_test = pickle.load( open( "../pickle/X_test.pkl", "rb" ) )
    y_test = pickle.load( open( "../pickle/y_test_hot.pkl", "rb" ) )


    

    # adding a singleton dimension and rescale to [0,1]
    X_train = np.asarray(np.expand_dims(X_train, axis=3))/float(255)
    X_val = np.asarray(np.expand_dims(X_val,axis=3))/float(255)
    X_test = np.asarray(np.expand_dims(X_test,axis=3))/float(255)
    
    X_train = np.float32(X_train)
    X_val = np.float32(X_val)
    X_test = np.float32(X_test)
    
    y_train = np.float32(y_train)
    y_val = np.float32(y_val)
    y_test = np.float32(y_test)
    
    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

In [6]:
# load the dataset
# the labels are one-hot-encoded

(X_train, y_train), (X_val, y_val), (X_test, y_test) = load_data()


In [7]:
# shape and type of the training set
print 'Training set  \t\t->\t', X_train.shape, y_train.shape
print 'Validation set \t\t->\t', X_val.shape, y_val.shape
print 'Test set \t\t->\t', X_test.shape, y_test.shape


Training set  		->	(12500, 32, 32, 1) (12500, 6)
Validation set 		->	(6250, 32, 32, 1) (6250, 6)
Test set 		->	(6250, 32, 32, 1) (6250, 6)


In [8]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])

# model definitions
    

In [9]:
def next_batch(batch_size):
    
    n_samples = len(X_train)
    indices = np.random.choice(n_samples, batch_size)
    batch_x = X_train[indices]
    batch_y = y_train[indices]
    
    return batch_x, batch_y

def conv2d(x, W, b, convLayer, strides=1):
    with tf.name_scope(convLayer):
        # Conv2D wrapper, with bias and relu activation
        x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME', name=convLayer)
        x = tf.nn.bias_add(x, b)
        return tf.nn.relu(x)


def maxpool2d(x, k):
    with tf.name_scope('AvgPooling'):
        # MaxPool2D wrapper
        return tf.nn.avg_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')
    
def batch_flatten(x):
    # Turn a n-D tensor into a 2D tensor where the first dimension is conserved.
    
    x = tf.reshape(x, [-1, np.prod(x.get_shape()[1:].as_list())])
    return x

In [10]:
# Parameters
image_size = 32
num_labels = 6
num_channels = 1 # grayscale

# model 
computer hangs

In [12]:
batch_size = 16
keep_prob = 1.0

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(X_val)
    tf_test_dataset = tf.constant(X_test)
    
    # variables
    wc1 = tf.Variable(tf.truncated_normal([2, 2, 1, 16], stddev=0.1))
    bc1 = tf.Variable(tf.zeros([16]))
    
    wc2 = tf.Variable(tf.truncated_normal([2, 2, 16, 36], stddev=0.1))
    bc2 = tf.Variable(tf.zeros([36]))
    
    wc3 = tf.Variable(tf.truncated_normal([2, 2, 36, 64], stddev=0.1))
    bc3 = tf.Variable(tf.zeros([64]))
    
    wc4 = tf.Variable(tf.truncated_normal([2, 2, 64, 100], stddev=0.1))
    bc4 = tf.Variable(tf.zeros([100]))
    
    wc5 = tf.Variable(tf.truncated_normal([2, 2, 100, 144], stddev=0.1))
    bc5 = tf.Variable(tf.zeros([144]))
    
    wd1 = tf.Variable(tf.random_normal([144, 864]))
    bd1 = tf.Variable(tf.random_normal([864]))
    
    wd2 = tf.Variable(tf.random_normal([864, 288]))
    bd2 = tf.Variable(tf.random_normal([288]))
    
    wd3 = tf.Variable(tf.random_normal([288, 6]))
    bd3 = tf.Variable(tf.random_normal([6]))
    
    
    # Model.
    def model(data):
        
        # Convolution Layer 1 : kernel 2x2, 1 channel, 16 features, strides = 1 
        
        conv1 = conv2d(data, wc1, bc1, 'convolution2D_1')
        
        # Convolution Layer 2
        
        conv2 = conv2d(conv1, wc2, bc2, 'convolution2D_2')
    
        # Convolution Layer 3
       
        conv3 = conv2d(conv2, wc3, bc3, 'convolution2D_1')
    
        # Convolution Layer 4
        
        conv4 = conv2d(conv3, wc4, bc4, 'convolution2D_1')
    
        # Convolution Layer 5
        
        conv5 = conv2d(conv4, wc5, bc5, 'convolution2D_1')
    
        #pool1 = maxpool2d(conv5, k=32)
        pool1 = tf.nn.avg_pool(conv5, ksize=[1, 32, 32, 1], strides=[1, 32, 32, 1], padding='SAME')
        
        # must do flattening and dropout first
        #pool1_reshaped = tf.squeeze(pool1, [1, 2])
        pool1_reshaped = batch_flatten(pool1)
        #pool1_dropout = tf.nn.dropout(pool1_reshaped, keep_prob) 
            
        h_fc1 = tf.nn.relu(tf.nn.bias_add(tf.matmul(pool1_reshaped, wd1), bd1))
        #h_fc1_dropout = tf.nn.dropout(h_fc1, keep_prob)
               
        h_fc2 = tf.nn.relu(tf.nn.bias_add(tf.matmul(h_fc1, wd2), bd2))
        #h_fc2_dropout = tf.nn.dropout(h_fc2, keep_prob)
               
        logits = tf.matmul(h_fc2, wd3) + bd3
        
        return logits 
    
    
    
    # Inference
    logits = model(tf_train_dataset)
   
    
    # Loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Training
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
     
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    # Validation set and Test set predictions
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    

# target model 
computer hangs

In [58]:
batch_size = 64
patch_size = 2
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(X_val)
    tf_test_dataset = tf.constant(X_test)
    
    # Variables.
    wc1 = tf.Variable(tf.truncated_normal([2, 2, 1, 16], stddev=0.1))
    bc1 = tf.Variable(tf.zeros([16]))
    
    wc2 = tf.Variable(tf.truncated_normal([2, 2, 16, 36], stddev=0.1))
    bc2 = tf.Variable(tf.zeros([36]))
    
    wc3 = tf.Variable(tf.truncated_normal([2, 2, 36, 64], stddev=0.1))
    bc3 = tf.Variable(tf.constant(1.0, shape=[64]))
    
    wc4 = tf.Variable(tf.truncated_normal([2, 2, 64, 100], stddev=0.1))
    bc4 = tf.Variable(tf.constant(1.0, shape=[100]))
    
    wc5 = tf.Variable(tf.truncated_normal([2, 2, 100, 144], stddev=0.1))
    bc5 = tf.Variable(tf.constant(1.0, shape=[144]))
    
    wd1 = tf.Variable(tf.truncated_normal([144, 864], stddev=0.1))
    bd1 = tf.Variable(tf.constant(1.0, shape=[864]))
    
    wd2 = tf.Variable(tf.truncated_normal([864, 288], stddev=0.1))
    bd2 = tf.Variable(tf.constant(1.0, shape=[288]))
    
    wd3 = tf.Variable(tf.truncated_normal([288, 6], stddev=0.1))
    bd3 = tf.Variable(tf.constant(1.0, shape=[6]))

    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, wc1, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + bc1)
        
        conv = tf.nn.conv2d(hidden, wc2, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + bc2)
        
        conv = tf.nn.conv2d(hidden, wc3, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + bc3)
        
        conv = tf.nn.conv2d(hidden, wc4, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + bc4)
        
        conv = tf.nn.conv2d(hidden, wc5, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + bc5)
        
        pool = tf.nn.avg_pool(hidden, ksize=[1, 32, 32, 1], strides=[1, 32, 32, 1], padding='SAME')
        
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        
        hidden = tf.nn.relu(tf.matmul(reshape, wd1) + bd1)
        
        hidden1 = tf.nn.relu(tf.matmul(hidden, wd2) + bc2)
        
        return tf.matmul(hidden1, wd3) + bd3
    
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

# simpler model with less feature maps per layer
works fine

In [11]:
batch_size = 16
patch_size = 2
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(X_val)
    tf_test_dataset = tf.constant(X_test)
    
    # Variables.
    wc1 = tf.Variable(tf.truncated_normal([2, 2, 1, 16], stddev=0.1))
    bc1 = tf.Variable(tf.zeros([16]))
    
    wc2 = tf.Variable(tf.truncated_normal([2, 2, 16, 36], stddev=0.1))
    bc2 = tf.Variable(tf.zeros([36]))
    
    wc3 = tf.Variable(tf.truncated_normal([2, 2, 36, 64], stddev=0.1))
    bc3 = tf.Variable(tf.constant(1.0, shape=[64]))
    
    wc4 = tf.Variable(tf.truncated_normal([2, 2, 64, 16], stddev=0.1))
    bc4 = tf.Variable(tf.constant(1.0, shape=[16]))
    
    wc5 = tf.Variable(tf.truncated_normal([2, 2, 16, 16], stddev=0.1))
    bc5 = tf.Variable(tf.constant(1.0, shape=[16]))
    
    wd1 = tf.Variable(tf.truncated_normal([16, 864], stddev=0.1))
    bd1 = tf.Variable(tf.constant(1.0, shape=[864]))
    
    wd2 = tf.Variable(tf.truncated_normal([864, 288], stddev=0.1))
    bd2 = tf.Variable(tf.constant(1.0, shape=[288]))
    
    wd3 = tf.Variable(tf.truncated_normal([288, 6], stddev=0.1))
    bd3 = tf.Variable(tf.constant(1.0, shape=[6]))

    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, wc1, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + bc1)
        
        conv = tf.nn.conv2d(hidden, wc2, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + bc2)
        
        conv = tf.nn.conv2d(hidden, wc3, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + bc3)
        
        conv = tf.nn.conv2d(hidden, wc4, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + bc4)
        
        conv = tf.nn.conv2d(hidden, wc5, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + bc5)
        
        pool = tf.nn.avg_pool(hidden, ksize=[1, 32, 32, 1], strides=[1, 32, 32, 1], padding='SAME')
        
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        
        hidden = tf.nn.relu(tf.matmul(reshape, wd1) + bd1)
        
        hidden1 = tf.nn.relu(tf.matmul(hidden, wd2) + bd2)
        
        return tf.matmul(hidden1, wd3) + bd3
    
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [12]:
def next_batch(batch_size):
    
    n_samples = len(X_train)
    indices = np.random.choice(n_samples, batch_size)
    batch_x = X_train[indices]
    batch_y = y_train[indices]
    
    return batch_x, batch_y

In [12]:
num_steps = 101

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        
        batch_data, batch_labels = next_batch(batch_size)
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 50 == 0):
            print 'step   \t\t->\t', step
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), y_val))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), y_test))

Initialized
step   		->	0
Minibatch loss at step 0: 7.236277
Minibatch accuracy: 31.2%
Validation accuracy: 36.3%
step   		->	50
Minibatch loss at step 50: 1.506475
Minibatch accuracy: 37.5%
Validation accuracy: 36.3%
step   		->	100
Minibatch loss at step 100: 1.444633
Minibatch accuracy: 37.5%
Validation accuracy: 36.3%
Test accuracy: 36.1%
